In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
from dotenv import load_dotenv
import edurel.utils.prompt_man as pm
import edurel.utils.conversation as conv
import edurel.utils.duckdb as ddb
import edurel.utils.er_agent as era
import edurel.utils.misc as misc
import yaml  

load_dotenv() 
BASE_DIR = os.getenv("BASE_DIR")
ER_DIR = f"{BASE_DIR}/er"
CONFIG_DIR = f"{BASE_DIR}/config"
LOG_DIR = f"{BASE_DIR}/logs/er"


/home/iclassen/.py1/lib/python3.14/site-packages/langchain_core/_api/deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


In [1]:
erm = """
entities:
- entityname: Building
  key: BuildingID
  attributes:
  - attributename: BuildingName
    type: TEXT
  - attributename: Address
    type: TEXT
  - attributename: City
    type: TEXT
  - attributename: PostalCode
    type: TEXT
  - attributename: ConstructionYear
    type: INTEGER
  - attributename: TotalFloors
    type: INTEGER
  - attributename: TotalArea
    type: DECIMAL

- entityname: Room
  key: RoomID
  attributes:
  - attributename: RoomNumber
    type: TEXT
  - attributename: RoomName
    type: TEXT
  - attributename: Floor
    type: INTEGER
  - attributename: Area
    type: DECIMAL
  - attributename: Capacity
    type: INTEGER

- entityname: RoomType
  key: RoomTypeID
  attributes:
  - attributename: TypeName
    type: TEXT
  - attributename: Description
    type: TEXT

- entityname: Amenity
  key: AmenityID
  attributes:
  - attributename: AmenityName
    type: TEXT
  - attributename: Description
    type: TEXT

associative_entities:
- associationname: RoomAmenity
  associations:
  - targetentity: Room
    role: Room
    cardinality: ONE
  - targetentity: Amenity
    role: Amenity
    cardinality: ONE
  identified_by:
  - targetentity: Room
    cardinality: ONE
  - targetentity: Amenity
    cardinality: ONE
  attributes:
  - attributename: Quantity
    type: INTEGER
  - attributename: InstallationDate
    type: DATE

relationships:
- relationshipname: Contains
  entities:
  - entityname: Building
    role: Building
  - entityname: Room
    role: Room
  cardinality:
    Building: ONE
    Room: MANY

- relationshipname: HasType
  entities:
  - entityname: Room
    role: Room
  - entityname: RoomType
    role: RoomType
  cardinality:
    Room: MANY
    RoomType: ONE

inheritances: []

valuelists:
- valuelistname: RoomStatus
- valuelistname: BuildingStatus
"""

In [4]:
schema = yaml.safe_load(erm)

In [6]:
mermaid_code = misc.er_to_mermaid(schema)                                                                                                                      
print(mermaid_code)  

erDiagram
    Building {
        string BuildingID PK
        TEXT BuildingName
        TEXT Address
        TEXT City
        TEXT PostalCode
        INTEGER ConstructionYear
        INTEGER TotalFloors
        DECIMAL TotalArea
    }
    Room {
        string RoomID PK
        TEXT RoomNumber
        TEXT RoomName
        INTEGER Floor
        DECIMAL Area
        INTEGER Capacity
    }
    RoomType {
        string RoomTypeID PK
        TEXT TypeName
        TEXT Description
    }
    Amenity {
        string AmenityID PK
        TEXT AmenityName
        TEXT Description
    }
    RoomStatus {
        INTEGER ID PK
        TEXT Description
        BOOLEAN IsValid
        INTEGER SortOrder
    }
    BuildingStatus {
        INTEGER ID PK
        TEXT Description
        BOOLEAN IsValid
        INTEGER SortOrder
    }
    RoomAmenity {
        INTEGER Quantity
        DATE InstallationDate
    }
    RoomAmenity }o--|| Room : "Room"
    RoomAmenity }o--|| Amenity : "Amenity"
    RoomAm

# Database

In [3]:
# db = ddb.Db.mem("company_en")
# db.config_exe(f"{CONFIG_DIR}/databases/company_en.yaml")

# Prompts

In [28]:
pman = pm.PromptMan(f"{ER_DIR}/building.txt")

In [30]:
eragent = era.ERAgent(
    conv.Client.STATS, 
    conv.LLMName.OPUS45,
    domain="building",
    log_dir=LOG_DIR,
)

# Testing

In [25]:
def run_er(pman, entry_id, autolog=True):
    eragent.prune_conversation(1)
    eragent.text2er(pman[entry_id][0], pman[entry_id][1], autolog=autolog)

In [ ]:
run_er(pman, 2)

In [32]:
for i in range(len(pman)):
    run_er(pman, i)